# 데이터프레임 합치기 (국회의원 별)

In [483]:
import json
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [484]:
# 데이터 불러오기
MP = pd.read_json('MP.json', orient = 'table')
PB = pd.read_json("PB.json", orient = 'table')

In [485]:
# 국회의원 별 df
MP

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
0,강기윤,국민의힘,[보건복지위원회],경남 창원시성산구,남,재선,지역구
1,강대식,국민의힘,"[국토교통위원회, 예산결산특별위원회]",대구 동구을,남,초선,지역구
2,강득구,더불어민주당,"[교육위원회, 국회운영위원회, 예산결산특별위원회]",경기 안양시만안구,남,초선,지역구
3,강민국,국민의힘,"[민생경제안정특별위원회, 정무위원회, 예산결산특별위원회]",경남 진주시을,남,초선,지역구
4,강민정,더불어민주당,"[중앙선거관리위원회위원(남래진)선출에관한인사청문특별위원회, 교육위원회, 국회운영...",비례대표,여,초선,비례대표
...,...,...,...,...,...,...,...
294,홍익표,더불어민주당,[문화체육관광위원회],서울 중구성동구갑,남,3선,지역구
295,홍정민,더불어민주당,"[산업통상자원중소벤처기업위원회, 여성가족위원회]",경기 고양시병,여,초선,지역구
296,황보승희,국민의힘,[문화체육관광위원회],부산 중구영도구,여,초선,지역구
297,황운하,더불어민주당,[정무위원회],대전 중구,남,초선,지역구


In [486]:
# 법률안 별 df
PB.head()

,법률안명,대표발의자,공동발의자,소관위원회,제안일,처리상태,상세페이지
0,성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김상훈, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, 조수진, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1,남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김승수, 배준영, 유경준, 유의동, 이명수, 정희용, 조수진, 지성호]",환경노동위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
2,양성평등기본법 | 일부개정법률안,강대식,"[김병욱, 김상훈, 김승수, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
3,여수ㆍ순천 10ㆍ19사건 진상규명 및 희생자 명예회복에 관한 특별법 | 일부개정법률안,소병철,"[김승남, 김정호, 김회재, 서동용, 이수진, 이용선, 정일영, 주철현, 한준호]",행정안전위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
4,조세특례제한법 | 일부개정법률안,정우택,"[김학용, 서정숙, 이양수, 이종성, 정운천, 정희용, 조명희, 지성호, 최승재, ...",기획재정위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...


## 법률안DF에서 의원명별로 새로운 DF 생성하기)

### 새로운 DF 생성

In [487]:
# 의원별로 새로운 df 생성 ('의원명', '발의자', '법률안명', '소관위원회')
person_li = MP['의원명'].tolist() + MP['의원명'].tolist() # 대표, 공동으로 구별하기 위해 두개씩 생성
person_se = pd.Series(person_li)

name_pb = pd.DataFrame(person_se, columns = ['의원명'])

#이름 오름차순 정렬
name_pb = name_pb.sort_values(['의원명'])

#인덱스 다시 설정
name_pb.reset_index(inplace=True, drop=True)

name_pb

,의원명
0,강기윤
1,강기윤
2,강대식
3,강대식
4,강득구
...,...
593,황보승희
594,황운하
595,황운하
596,황희


In [488]:
type(name_pb)

pandas.core.frame.DataFrame

In [489]:
# 발의자 설정(대표/ 공동)
name_pb['발의자'] = '대표'
name_pb.iloc[1::2,1] = '공동'

# 법률안명, 소관위원회 추가(리스트 형식으로)
li = []
for _ in range(598) : li.append([])

In [490]:
name_pb['법률안명'] = pd.Series(li)
name_pb['소관위원회'] = pd.Series(li)
name_pb

,의원명,발의자,법률안명,소관위원회
0,강기윤,대표,[],[]
1,강기윤,공동,[],[]
2,강대식,대표,[],[]
3,강대식,공동,[],[]
4,강득구,대표,[],[]
...,...,...,...,...
593,황보승희,공동,[],[]
594,황운하,대표,[],[]
595,황운하,공동,[],[]
596,황희,대표,[],[]


In [491]:
df = name_pb.copy()

### 법률안명, 소관위원회 값 추가

- 대표자일 때 

In [492]:
#예시
ps = '강대식'
for i in range(4):
    idx = df[(df['의원명'] == ps)&(df['발의자'] == '대표')].index

    old = list(df.loc[idx, '법률안명']) #name_pb의 법률안명 컬럼
    add = PB.loc[i, '법률안명'] #pb의 법률안명 컬럼
    new = [old[0] + [add]] #[[a,b]]로 0번째 인덱스 값만 추출
    
    df.loc[idx, '법률안명'] = new
list(df.loc[idx, '법률안명'])

[['성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안',
  '남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안',
  '양성평등기본법 | 일부개정법률안',
  '여수ㆍ순천 10ㆍ19사건 진상규명 및 희생자 명예회복에 관한 특별법 | 일부개정법률안']]

In [493]:
PB2 = PB.head(690) #691이상부터 list index out of range error 발생

In [494]:
PB.loc[690, :]

법률안명                                             한·중남미재단법안
대표발의자                                                  이광재
공동발의자    [김상희, 김영배, 김영주, 문진석, 오영환, 이용빈, 이용선, 이재정, 태영호, ...
소관위원회                                              외교통일위원회
제안일                                    2022-04-28 00:00:00
처리상태                                                   NaN
상세페이지    http://likms.assembly.go.kr/bill/billDetail.do...
Name: 690, dtype: object

In [506]:
#여기때문에 오류 발생
df[(df['의원명'] == '')&(df['발의자'] == '대표')]

,의원명,발의자,법률안명,소관위원회
2,강대식,대표,"[성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안, 남녀고용평등과 일·가...","[여성가족위원회, 환경노동위원회, 여성가족위원회, 행정안전위원회, 정무위원회, 행정..."


In [496]:
# 대표발의자 일 때 법률안명 (index에러 예외처리)

try:
    for i, ps in enumerate(PB['대표발의자']):
    
        idx = df[(df['의원명'] == ps)&(df['발의자'] == '대표')].index
    
        old = list(df.loc[idx, '법률안명']) #name_pb의 법률안명 컬럼
        add = PB.loc[i, '법률안명'] #pb의 법률안명 컬럼
        new = [old[0] + [add]] #[[a,b]]로 0번째 인덱스 값만 추출

        df.loc[idx, '법률안명'] = new
except IndexError :
    pass
    
df.head()

,의원명,발의자,법률안명,소관위원회
0,강기윤,대표,"[감염병의 예방 및 관리에 관한 법률 | 일부개정법률안, 아동복지법 | 일부개정법률...",[]
1,강기윤,공동,[],[]
2,강대식,대표,"[성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안, 남녀고용평등과 일·가...",[]
3,강대식,공동,[],[]
4,강득구,대표,"[평생교육법 | 일부개정법률안, 초·중등교육법 | 일부개정법률안, 학교급식법 | 일...",[]


In [497]:
# 대표발의자 일 때 소관위원회
try:
    
    for i, ps in enumerate(PB['대표발의자']):

        idx = df[(df['의원명'] == ps)&(df['발의자'] == '대표')].index

        old = list(df.loc[idx, '소관위원회']) #name_pb의 법률안명 컬럼
        add = PB.loc[i, '소관위원회'] #pb의 법률안명 컬럼
        new = [old[0] + [add]] #[[a,b]]로 0번째 인덱스 값만 추출

    #     print(new)

        df.loc[idx,'소관위원회'] = new

except IndexError :
    pass
df.head()

,의원명,발의자,법률안명,소관위원회
0,강기윤,대표,"[감염병의 예방 및 관리에 관한 법률 | 일부개정법률안, 아동복지법 | 일부개정법률...","[보건복지위원회, 보건복지위원회, 보건복지위원회]"
1,강기윤,공동,[],[]
2,강대식,대표,"[성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안, 남녀고용평등과 일·가...","[여성가족위원회, 환경노동위원회, 여성가족위원회, 행정안전위원회, 정무위원회, 행정..."
3,강대식,공동,[],[]
4,강득구,대표,"[평생교육법 | 일부개정법률안, 초·중등교육법 | 일부개정법률안, 학교급식법 | 일...","[교육위원회, 교육위원회, 교육위원회]"


- 공동발의자일 때

In [498]:
# 예시
#for ind, i in enumerate(PB['공동발의자']):
PB2 = PB.copy()
df1 = df.copy()
ind = 1
i = PB2['공동발의자'][ind]

for ps in PB2['공동발의자'][ind]:
    
    idx = df1[(df1['의원명'] == ps)&(df1['발의자'] == '공동')].index
    
    print(idx)
    before = list(df1.loc[idx, '법률안명'])
    add = PB2.loc[ind, '법률안명']
    after = [before[0] + [add]]
    
    df1.loc[idx, '법률안명'] = after

df1[91:526]

Int64Index([67, 69], dtype='int64')
Int64Index([91], dtype='int64')
Int64Index([221], dtype='int64')
Int64Index([319], dtype='int64')
Int64Index([327], dtype='int64')
Int64Index([371], dtype='int64')
Int64Index([499], dtype='int64')
Int64Index([505], dtype='int64')
Int64Index([525], dtype='int64')


,의원명,발의자,법률안명,소관위원회
91,김승수,공동,[남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안],[]
92,김승원,대표,[공직선거법 | 일부개정법률안],[행정안전위원회]
93,김승원,공동,[],[]
94,김영배,대표,[기부금품의 모집 및 사용에 관한 법률 | 일부개정법률안],[행정안전위원회]
95,김영배,공동,[],[]
...,...,...,...,...
521,주철현,공동,[],[]
522,주호영,대표,[],[]
523,주호영,공동,[],[]
524,지성호,대표,[경찰관 직무집행법 | 일부개정법률안],[행정안전위원회]


In [499]:
#예시2
PB3 = PB2.copy()
df2 = df1.copy()
ind = 2
i = PB3['공동발의자'][ind]

for ps in PB3['공동발의자'][ind]:
    
    idx = df2[(df2['의원명'] == ps)&(df2['발의자'] == '공동')].index
    
    print(idx)
    before = list(df2.loc[idx, '법률안명'])
    add = PB3.loc[ind, '법률안명']
    after = [before[0] + [add]]
    
    df2.loc[idx, '법률안명'] = after

df2[91:526]

Int64Index([67, 69], dtype='int64')
Int64Index([73], dtype='int64')
Int64Index([91], dtype='int64')
Int64Index([221], dtype='int64')
Int64Index([263], dtype='int64')
Int64Index([319], dtype='int64')
Int64Index([327], dtype='int64')
Int64Index([369], dtype='int64')
Int64Index([371], dtype='int64')
Int64Index([505], dtype='int64')
Int64Index([525], dtype='int64')


,의원명,발의자,법률안명,소관위원회
91,김승수,공동,"[남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안, 양성평등기본법 ...",[]
92,김승원,대표,[공직선거법 | 일부개정법률안],[행정안전위원회]
93,김승원,공동,[],[]
94,김영배,대표,[기부금품의 모집 및 사용에 관한 법률 | 일부개정법률안],[행정안전위원회]
95,김영배,공동,[],[]
...,...,...,...,...
521,주철현,공동,[],[]
522,주호영,대표,[],[]
523,주호영,공동,[],[]
524,지성호,대표,[경찰관 직무집행법 | 일부개정법률안],[행정안전위원회]


In [500]:
# 공동발의자 일 때 법률안명
try:
    for ind, i in enumerate(PB['공동발의자']):

        for ps in PB['공동발의자'][ind]: #해당 행에 이름이 있을 때
            # name_pb에서 의원명이 있는 인덱스 번호 받기 
            idx = df[(df['의원명'] == ps)&(df['발의자'] == '공동')].index

            #원래 데이터에 새로 추가
            before = list(df.loc[idx, '법률안명'])
            add = PB.loc[ind, '법률안명']
            after = [before[0] + [add]]

            df.loc[idx, '법률안명'] = after
except IndexError :
    pass

df

,의원명,발의자,법률안명,소관위원회
0,강기윤,대표,"[감염병의 예방 및 관리에 관한 법률 | 일부개정법률안, 아동복지법 | 일부개정법률...","[보건복지위원회, 보건복지위원회, 보건복지위원회]"
1,강기윤,공동,"[중소기업기본법 | 일부개정법률안, 형법 | 일부개정법률안, 조세특례제한법 | 일부...",[]
2,강대식,대표,"[성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안, 남녀고용평등과 일·가...","[여성가족위원회, 환경노동위원회, 여성가족위원회, 행정안전위원회, 정무위원회, 행정..."
3,강대식,공동,"[상표법 | 일부개정법률안, 종주국 김치, 인삼, 태권도의 세계화 지원에 관한 | ...",[]
4,강득구,대표,"[평생교육법 | 일부개정법률안, 초·중등교육법 | 일부개정법률안, 학교급식법 | 일...","[교육위원회, 교육위원회, 교육위원회]"
...,...,...,...,...
593,황보승희,공동,"[원자력안전위원회의 설치 및 운영에 관한 법률 | 일부개정법률안, 원자력안전법 | ...",[]
594,황운하,대표,[],[]
595,황운하,공동,"[모자보건법 | 일부개정법률안, 비송사건절차법 | 일부개정법률안, 국세기본법 | 일...",[]
596,황희,대표,[],[]


In [502]:
# 공동발의자 일 때 소관위원회
try:
    for ind, i in enumerate(PB['공동발의자']):

        for ps in PB['공동발의자'][ind]: #해당 행에 이름이 있을 때
            # name_pb에서 의원명이 있는 인덱스 번호 받기 
            idx = df[(df['의원명'] == ps)&(df['발의자'] == '공동')].index

            #원래 데이터에 새로 추가
            before = list(df.loc[idx, '소관위원회'])
            add = PB.loc[ind, '소관위원회']
            after = [before[0] + [add]]

            df.loc[idx, '소관위원회'] = after
except IndexError :
    pass
df

,의원명,발의자,법률안명,소관위원회
0,강기윤,대표,"[감염병의 예방 및 관리에 관한 법률 | 일부개정법률안, 아동복지법 | 일부개정법률...","[보건복지위원회, 보건복지위원회, 보건복지위원회]"
1,강기윤,공동,"[중소기업기본법 | 일부개정법률안, 형법 | 일부개정법률안, 조세특례제한법 | 일부...","[산업통상자원중소벤처기업위원회, 법제사법위원회, 기획재정위원회, 환경노동위원회, 행..."
2,강대식,대표,"[성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안, 남녀고용평등과 일·가...","[여성가족위원회, 환경노동위원회, 여성가족위원회, 행정안전위원회, 정무위원회, 행정..."
3,강대식,공동,"[상표법 | 일부개정법률안, 종주국 김치, 인삼, 태권도의 세계화 지원에 관한 | ...","[산업통상자원중소벤처기업위원회, 문화체육관광위원회, 교육위원회, 기획재정위원회, 문..."
4,강득구,대표,"[평생교육법 | 일부개정법률안, 초·중등교육법 | 일부개정법률안, 학교급식법 | 일...","[교육위원회, 교육위원회, 교육위원회]"
...,...,...,...,...
593,황보승희,공동,"[원자력안전위원회의 설치 및 운영에 관한 법률 | 일부개정법률안, 원자력안전법 | ...","[과학기술정보방송통신위원회, 과학기술정보방송통신위원회, 국방위원회, 교육위원회, 산..."
594,황운하,대표,[],[]
595,황운하,공동,"[모자보건법 | 일부개정법률안, 비송사건절차법 | 일부개정법률안, 국세기본법 | 일...","[보건복지위원회, 법제사법위원회, 기획재정위원회, 기획재정위원회, 농림축산식품해양수..."
596,황희,대표,[],[]


In [503]:
# 없는 값 확인
PB[(PB['대표발의자']=='황운하')|(PB['대표발의자']=='황희')] #왜...?ㅠㅠ 왜.....

,법률안명,대표발의자,공동발의자,소관위원회,제안일,처리상태,상세페이지
872,국가공무원법 | 일부개정법률안,황운하,"[김남국, 김두관, 김병기, 양정숙, 오영환, 이수진, 정일영, 정태호, 홍기원, ...",행정안전위원회,2022-04-13,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1183,변호사법 | 일부개정법률안,황운하,"[김남국, 김두관, 김병기, 신정훈, 안호영, 오영환, 조정훈, 한준호, 홍정민]",법제사법위원회,2022-03-18,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1202,광업법 | 일부개정법률안,황운하,"[고영인, 김경협, 김두관, 김승원, 김용민, 민병덕, 민형배, 양정숙, 오영환, ...",산업통상자원중소벤처기업위원회,2022-03-16,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1449,공직선거법 | 일부개정법률안,황운하,"[강민정, 김승원, 김용민, 민형배, 오영환, 유정주, 윤영덕, 장경태, 전용기, ...",행정안전위원회,2022-01-27,NaN,http://likms.assembly.go.kr/bill/billDetail.do...


In [504]:
df[(df['의원명']=='황운하')|(df['의원명']=='황희')]

,의원명,발의자,법률안명,소관위원회
594,황운하,대표,[],[]
595,황운하,공동,"[모자보건법 | 일부개정법률안, 비송사건절차법 | 일부개정법률안, 국세기본법 | 일...","[보건복지위원회, 법제사법위원회, 기획재정위원회, 기획재정위원회, 농림축산식품해양수..."
596,황희,대표,[],[]
597,황희,공동,[항만공사법 | 일부개정법률안],[농림축산식품해양수산위원회]


### 소관위원회 정리 


In [167]:
li = ('a', 'b', 'b', 'b', 'a')
li2 = ('a', 'c', 'c', 'b', 'a')
li_s = pd.Series(li)
li2_s = pd.Series(li2)

df2 = pd.concat([li_s, li2_s], axis=1)
df2['3'] = li_s + li2_s + 'c' + 'd' + 'c' + 'e'

In [168]:
df2

,0,1,3
0,a,a,aacdce
1,b,c,bccdce
2,b,c,bccdce
3,b,b,bbcdce
4,a,a,aacdce


In [169]:
df2[0]

0    a
1    b
2    b
3    b
4    a
Name: 0, dtype: object

In [170]:
df2.loc[0,'3'][0]

'a'

In [171]:
df2['3'][0][1]

'a'

In [172]:
df2['4'] = ''

In [173]:
df2['3']

0    aacdce
1    bccdce
2    bccdce
3    bbcdce
4    aacdce
Name: 3, dtype: object

In [ ]:
#빈 열 생성
df['소관위원회_count'] = ''

#딕셔너리 대입


In [209]:
#빈 열 생성
df2['4'] = ''

In [211]:
#딕셔너리 대입
for i in range(5):
    df2['4'][i]= set(df2['3'][i])
df2['4']

0    {a, c, d, e}
1    {c, d, e, b}
2    {c, d, e, b}
3    {c, d, e, b}
4    {a, c, d, e}
Name: 4, dtype: object

In [212]:
#다시 리스트로 변환
df2['4'] = list(df2['4'])
text_li

['c', 'd', 'e', 'b']

In [214]:
for i in range(5):
    df2['4'][i] = {text : 0 for text in df2['4'][i]}
df2

,0,1,3,4
0,a,a,aacdce,"{'a': 0, 'c': 0, 'd': 0, 'e': 0}"
1,b,c,bccdce,"{'c': 0, 'd': 0, 'e': 0, 'b': 0}"
2,b,c,bccdce,"{'c': 0, 'd': 0, 'e': 0, 'b': 0}"
3,b,b,bbcdce,"{'c': 0, 'd': 0, 'e': 0, 'b': 0}"
4,a,a,aacdce,"{'a': 0, 'c': 0, 'd': 0, 'e': 0}"


In [219]:
for j in range(5):
    for i in df2['3'][j]:
        
        df2.loc[j,'4'][i] = df2['3'][j].count[i]
        
df2['4']

TypeError: 'builtin_function_or_method' object is not subscriptable

## count 열 추가

In [ ]:
len(각 법률안 명)
# 대표 몇번 ()
# 공동 몇번 ()

## 데이터프레임 합치기

In [ ]:
PB.join(MB)